In [1]:
import geopandas as gpd
import folium
# from folium.plugins import HeatMap
from folium import plugins
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
un_schools_df = "data/ml-validation-2nd_iteration/ml_unrecognized_school_tiles_points_all.geojson"

In [3]:
def gen_heatmap(geojson, zoom_start=5, max_len = 54000, pt_radius=15, \
                draw_heatmap=False, heat_map_weights_col=None, \
                heat_map_weights_normalize=True, heat_map_radius=15):
    """
    Generate a heatmap from human mapper validated ML prediction for unrecognized schools

    Arg:
        geojson: geojson that contained centroid points of each unrecoginzed school tiles.
        zoom_start: Integer representing the initial zoom of the map
        plot_points: Add points to map (boolean)
        pt_radius: Size of each point
        draw_heatmap: Add heatmap to map (boolean)
        heat_map_weights_col: Column containing heatmap weights
        heat_map_weights_normalize: Normalize heatmap weights (boolean)
        heat_map_radius: Size of heatmap point

    Returns:
        folium map object
    """

    ## read in centroid points
    points = gpd.read_file(geojson)
    points['latitude'] = points.geometry.y
    points['longitude'] = points.geometry.x

    
    # get the median point of the maps
    middle_lat = points['latitude'].median()
    middle_lon = points['longitude'].median()

    
    curr_map = folium.Map(location=[middle_lat, middle_lon],
                          zoom_start=zoom_start)


    # add heatmap
    if draw_heatmap:
        # convert to (n, 2) or (n, 3) matrix format
        if heat_map_weights_col is None:
            cols_to_pull = ["latitude", "longitude"]
        else:
            # if we have to normalize
            if heat_map_weights_normalize:
                points[heat_map_weights_col] = points[heat_map_weights_col] *100

            cols_to_pull = ["latitude", "longitude", heat_map_weights_col]
        
        if len(points) >= max_len:
            unreg_schools = points[cols_to_pull][:max_len].as_matrix()
        else:
            unreg_schools = unreg_schools

        curr_map.add_child(plugins.HeatMap(unreg_schools, radius=heat_map_radius))
        curr_map.save("unrecognized_school_heatmap.html")
        

    return curr_map

In [6]:
#if you want the interactive map show up in this notebook
# try to set the max_len = 54000
gen_heatmap(un_schools_df, zoom_start=5, max_len = 60568, pt_radius=15,draw_heatmap=True, heat_map_weights_normalize=True, heat_map_radius = 15)